### HTML Report

Import library and read local cpu_ffmpeg.csv

In [1]:

import pandas as pd
import numpy as np
import importlib
from scipy import stats
from skfeature.utility import *
from sklearn.feature_selection import VarianceThreshold, mutual_info_classif, SelectKBest, chi2, f_classif


In [2]:

pwd = "C:/Users/42times28/Desktop/AI506/AMD/Assignment2/"
inputDir = "Input/"
processedDir = "Preprocessed/"
featuresDir = "Features/"
#filename = "cpu-data.csv"
filename = "df_latest_nj_8bench_mem.xlsx"

In [3]:
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [4]:

from bokeh.io import show
from bokeh.palettes import Viridis256, inferno
from bokeh.plotting import figure, output_file, show
from bokeh.models import CustomJS, ColumnDataSource, BasicTicker, ColorBar, LinearColorMapper, Whisker, TabPanel, Tabs
from bokeh.transform import cumsum, factor_cmap, transform
from bokeh.models.widgets import Select, DataTable, DateFormatter, TableColumn, Div, Paragraph, PreText
from bokeh.layouts import column, row


In [5]:

def interactive_correlation_heatmap(dataframes, benchmarks, features, output_file_name = "", size=1000):
    data = []
    titles = benchmarks
    corrmat = []
    for i in range(len(dataframes)):
        dataframe = dataframes[i]
        corrmat = dataframe.corr()
        corrmat.index.name = 'AllColumns1'
        corrmat.columns.name = 'AllColumns2'
        corrmat = corrmat.stack().rename("value").reset_index()
        data.append(corrmat)

    sources = ColumnDataSource(data[0])

    # I am using 'Viridis256' to map colors with value, change it with 'colors' if you need some specific colors
    mapper = LinearColorMapper(
        palette=Viridis256, low=-1, high=1)

    # Define a figure and tools
    TOOLS = "box_select,lasso_select,pan,wheel_zoom,box_zoom,reset,help, hover"
    p = figure(
        tools=TOOLS,
        tooltips="@value",
        width=size,
        height=size - 200,
        title="Correlation plot",
        x_range=list(corrmat.AllColumns1.drop_duplicates()),
        y_range=list(corrmat.AllColumns2.drop_duplicates()),
        toolbar_location="right",
        x_axis_location="below")

    # Create rectangle for heatmap
    p.rect(
        x="AllColumns1",
        y="AllColumns2",
        width=1,
        height=1,
        source=sources,
        line_color=None,
        fill_color=transform('value', mapper))

    # Add legend
    color_bar = ColorBar(
        color_mapper=mapper,
        location=(0, 0),
        ticker=BasicTicker(desired_num_ticks=10))

    p.xaxis.major_label_orientation = "vertical"
    p.add_layout(color_bar, 'right')

    data = [i.to_dict('list') for i in data]
    data = dict(zip(titles, data))

    select = Select(title="Benchmarks: ", value=titles[0], options=titles)

    select.js_on_change("value", CustomJS(args=dict(sources=sources, data=data), code="""
        //console.log(sources.data);
        //console.log(data)
        sources.data = data[cb_obj.value];
        //console.log(sources.data);
    """))

    
    if output_file_name != "":
        show(column(p, select))
        output_file(output_file_name + ".html")
    
    return TabPanel(child=column(p, select), title="Correlations")

In [6]:


def interactive_scatterplot(dataframes, benchmarks, features, output_file_name = "", size=1000):
    titles = benchmarks
    data = []
    
    for i in range(len(dataframes)):
        df_zscore = dataframes[i].apply(stats.zscore)        #df_zscore = (df - df.mean())/df.std()
        sample = df_zscore.sample(1000)
        feature_data = []
        for j in range(len(features)):
            feature_data.append(sample[features[j]])
        data.append(dict(zip(features, feature_data)))
    data = dict(zip(titles, data))

    sources = ColumnDataSource(data=dict(x = data[titles[0]][features[0]], y = data[titles[0]][features[0]]))

    p = figure(width=size, 
               height=size-200, 
               title = "Standard Deviation Plot")
    p.scatter('x', 'y', source=sources)

    select_benchmark = Select(title="Benchmarks: ", value=titles[0], options=titles)
    select_feature_x = Select(title="Featrues x: ", value=features[0], options=features) 
    select_feature_y = Select(title="Featrues y: ", value=features[0], options=features) 
    
    select_benchmark.js_on_change("value", CustomJS(args=dict(select_feature_x=select_feature_x, select_feature_y=select_feature_y, sources=sources, data=data), code="""
        //console.log(sources);
        //console.log(data)
        const obj = { x: data[cb_obj.value][select_feature_x.value], y: data[cb_obj.value][select_feature_y.value] };
        sources.data = obj;
        //console.log(x);
    """))
    
    select_feature_x.js_on_change("value", CustomJS(args=dict(select_benchmark=select_benchmark, select_feature_y=select_feature_y, sources=sources, data=data), code="""
        console.log(sources.data.x);
        console.log(data[select_benchmark.value][cb_obj.value]);
        const obj = { x: data[select_benchmark.value][cb_obj.value], y: data[select_benchmark.value][select_feature_y.value] };
        //sources.data = obj;
        //console.log(x);
    """))
    
    select_feature_y.js_on_change("value", CustomJS(args=dict(select_benchmark=select_benchmark, select_feature_x=select_feature_x, sources=sources, data=data), code="""
        //console.log(sources.data.x);
        //console.log(data[select_benchmark.value][select_feature_x.value])
        const obj = { x: data[select_benchmark.value][select_feature_x.value], y: data[select_benchmark.value][cb_obj.value] };
        sources.data = obj;
        //console.log(x);
    """))

    if output_file_name != "":
        show(column(p, select_benchmark, select_feature))
        output_file(output_file_name + ".html")
    return TabPanel(child=column(p, select_benchmark, row(select_feature_x, select_feature_y)), title="Scatters")



In [7]:

def interactive_histgram(dataframes, benchmarks, features, output_file_name = "", size=1000):
    titles = benchmarks
    features = features
    data = []

    for i in range(len(dataframes)):
        data_features = []
        df = dataframes[i]
        for j in features:
            # Histogram
            x = df[j]
            bins = np.linspace(x.min(), x.max(), 40)
            hist, edges = np.histogram(x, density=True, bins=bins)
            data_features.append(dict(top=hist, bottom=[0]* len(hist), left=edges[:-1], right=edges[1:]))
        data.append(dict(zip(features, data_features)))


    data = dict(zip(titles, data))
    TOOLS = "box_select,lasso_select,pan,wheel_zoom,box_zoom,reset,help, hover"
    p = figure(width=size, 
               height=size-200, 
               tools=TOOLS,
               title="Histogram")

    x = data[titles[0]][features[0]]
    x = ColumnDataSource(x)
    p.quad(source=x,
             fill_color="skyblue", line_color="white",
             legend_label="samples")

    p.y_range.start = 0
    p.xaxis.axis_label = "x"
    p.yaxis.axis_label = "Counts(x)"

    select_benchmark = Select(title="Benchmarks: ", value=titles[0], options=titles)
    select_feature = Select(title="Featrues : ", value=features[0], options=features) 

    select_benchmark.js_on_change("value", CustomJS(args=dict(select_feature=select_feature, x=x, data=data), code="""
        //console.log(x);
        x.data = data[cb_obj.value][select_feature.value];
        //console.log(x);
    """))


    select_feature.js_on_change("value", CustomJS(args=dict(select_benchmark=select_benchmark, x=x, data=data), code="""
        //console.log(data[select_benchmark.value][cb_obj.value]);
        x.data = data[select_benchmark.value][cb_obj.value];
        //console.log(x);
    """))
    
    
    if output_file_name != "":
        show(column(p, select_benchmark, select_feature))
        output_file(output_file_name + ".html")
    
    return TabPanel(child=column(p, select_benchmark, select_feature), title="Histgrams")


In [8]:

def interactive_boxplot(dataframes, benchmarks, features, output_file_name = "", size=1000, log_transformation = False):
    titles = benchmarks
    data = []
    data_outliers = []
    df = pd.DataFrame()
    features = features
    for i in range(len(dataframes)):
        if log_transformation:
            df = np.log10(dataframes[i])
        else:
            df = dataframes[i]
        # compute quantiles
        qs = df.quantile([0.25, 0.5, 0.75], axis=0).T
        qs.columns = ["q1", "q2", "q3"]
        
        # compute IQR outlier bounds
        iqr = qs.q3 - qs.q1
        qs["upper"] = qs.q3 + 1.5*iqr
        qs["lower"] = qs.q1 - 1.5*iqr

        data_out_iqr = df[df.lt(qs["lower"].T, axis=1) | df.ge(qs["upper"].T, axis=1)]
        data_outliers_in_feature = pd.DataFrame(columns=['value', 'feature'])
        df_temp = pd.DataFrame()

        for j in features:
            df_temp['value'] = data_out_iqr[j]
            df_temp['feature'] = j
            data_outliers_in_feature = pd.concat([data_outliers_in_feature, df_temp.dropna()])

        data_outliers.append(data_outliers_in_feature.to_dict('list'))
        df = df.reset_index(drop=True)
        df = pd.concat([df, qs.T])

        df = df.T
        df = df.reset_index()
        df.columns.values[0] = "feature"
        df.columns = df.columns.map(str)
        data.append(df.to_dict('list'))

    outliers = dict(zip(titles, data_outliers))
    data = dict(zip(titles, data))

    sources_outlier = ColumnDataSource(data=outliers[titles[0]])
    sources = ColumnDataSource(data=data[titles[0]])
    TOOLS = "box_select,lasso_select,pan,wheel_zoom,box_zoom,reset,help, hover"
    p = figure(x_range=features, 
           tools=TOOLS,
           title="BoxPlot", 
           width=size, 
           height=size-200,
           background_fill_color="#eaefef", 
           y_axis_label=  "Log-10 Range" if log_transformation else "Range")
    # outlier range
    whisker = Whisker(base="feature", upper="upper", lower="lower", source=sources, line_width=1)
    whisker.upper_head.size = whisker.lower_head.size = 20
    p.add_layout(whisker)

    # quantile boxes
    cmap = factor_cmap("feature", inferno(len(features)), features)
    p.vbar("feature", 0.7, "q2", "q3", source=sources, color=cmap, line_color="black")
    p.vbar("feature", 0.7, "q1", "q2", source=sources, color=cmap, line_color="black")

    # outliers
    p.scatter("feature", "value", source=sources_outlier, size=6, color="black", alpha=0.3)
    
    p.xaxis.major_label_orientation = "vertical"
    #p.xgrid.grid_line_color = None
    #p.axis.major_label_text_font_size="14px"
    #p.axis.axis_label_text_font_size="12px"


    select_benchmark = Select(title="Benchmarks: ", value=titles[0], options=titles)

    select_benchmark.js_on_change("value", CustomJS(args=dict(sources=sources, data=data, sources_outlier=sources_outlier, outliers=outliers), code="""
        //console.log(data);
        //console.log(sources)
        sources.data = data[cb_obj.value];
        sources_outlier.data = outliers[cb_obj.value];
        //console.log(sources);
    """))

    
    if output_file_name != "":
        show(column(p, select_benchmark))
        output_file(output_file_name + ".html")

    return TabPanel(child=column(p, select_benchmark), title="Boxplots")


In [9]:
def load_excel_to_dataframe(filename = ""):
    if filename == "":
        from tkinter import Tk
        from tkinter.filedialog import askopenfilename

        Tk().withdraw()
        filename = askopenfilename()
        
    df = pd.read_excel(filename)
    return df

In [10]:
def drop_string_datetime_na(dataframe):
    dataframe = dataframe[list(dataframe.T[(dataframe.dtypes==np.float64) | (dataframe.dtypes==np.int64)].index) + ["host", "uuid"]]
    dataframe[dataframe.T[dataframe.dtypes==np.int64].index] = dataframe[dataframe.T[dataframe.dtypes==np.int64].index].astype(float)
    dataframe = dataframe.dropna(axis=1, how='all')
    nunique = dataframe.nunique()
    cols_to_drop = nunique[nunique == 1].index
    dataframe = dataframe.drop(cols_to_drop + ["Unnamed: 0"], axis=1)
    return dataframe

In [12]:

def removeCorrelatedFeatures(dataframe):
    sel = VarianceThreshold(threshold=0.01)
    sel.fit_transform(dataframe)
    quasi_constant = [col for col in dataframe.columns if col not in sel.get_feature_names_out()]
    train = dataframe[sel.get_feature_names_out()]
    corr_matrix = train.corr()
    corr_features = [feature for feature in corr_matrix.columns if (corr_matrix[feature].iloc[:corr_matrix.columns.get_loc(feature)] > 0.8).any()]
    return dataframe.drop(quasi_constant + corr_features, axis=1)


def mutualInformationChart(dataframe, y, selectedFeature, size = 400):
    mutual_features = SelectKBest(mutual_info_classif, k = len(dataframe.columns) // 4 + 2)
    X_kbest_features = mutual_features.fit(dataframe.to_numpy(), y)
    selectedFeature += X_kbest_features.get_feature_names_out(dataframe.columns).tolist()
    
    importances = mutual_info_classif(dataframe.to_numpy(), y)
    
    p = figure(x_range=list(dataframe.columns), width=size, 
               height=size+200, title="Mutual Infomation",
               toolbar_location=None, tools="")

    p.vbar(x=dataframe.columns, top=importances, width=0.9)
    p.xaxis.major_label_orientation = "vertical"
    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    
    return p
    
def chiSquareChart(dataframe, y, selectedFeature, size = 400):
    # k features with highest chi-squared statistics are selected
    chi2_features = SelectKBest(chi2, k = len(dataframe.columns) // 4 + 2)
    X_kbest_features = chi2_features.fit(dataframe.to_numpy(), y)
    selectedFeature += X_kbest_features.get_feature_names_out(dataframe.columns).tolist()
    
    importances = chi2(dataframe.to_numpy(), y)
    
    p = figure(x_range=list(dataframe.columns), width=size, 
               height=size+200, title="Chi Square",
               toolbar_location=None, tools="")

    p.vbar(x=dataframe.columns, top=importances, width=0.9)
    p.xaxis.major_label_orientation = "vertical"
    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    
    return p
    
def fScoreChart(dataframe, y, selectedFeature, size = 400):
    # k features with highest f-score statistics are selected
    f_classif_features = SelectKBest(f_classif, k = len(dataframe.columns) // 4 + 2)
    X_kbest_features = f_classif_features.fit(dataframe.to_numpy(), y)
    selectedFeature += X_kbest_features.get_feature_names_out(dataframe.columns).tolist()
    
    importances = f_classif(dataframe.to_numpy(), y)
    
    p = figure(x_range=list(dataframe.columns), width=size, 
               height=size+200, title="F score",
               toolbar_location=None, tools="")

    p.vbar(x=dataframe.columns, top=importances, width=0.9)
    p.xaxis.major_label_orientation = "vertical"
    p.xgrid.grid_line_color = None
    p.y_range.start = 0

    return p

def selectFeaturesChart(dataframe, y, selectedFeature, function_name, size = 400):
    module = importlib.import_module('sklearn.feature_selection')
    func = getattr(module, function_name)
    # k features with highest f-score statistics are selected
    func_features = SelectKBest(func, k = len(dataframe.columns) // 4 + 2)
    X_kbest_features = func_features.fit(dataframe.to_numpy(), y)
    selectedFeature += X_kbest_features.get_feature_names_out(dataframe.columns).tolist()
    
    importances = np.array(func(dataframe.to_numpy(), y))
    
    TOOLS = "box_select,lasso_select,pan,wheel_zoom,box_zoom,reset,help, hover"
    p = figure(x_range=list(dataframe.columns), width=size, 
               height=size+200, title=function_name,
               toolbar_location=None, tools=TOOLS)

    p.vbar(x=dataframe.columns, top=importances.flatten()[0:len(dataframe.columns)], width=0.9)
    p.xaxis.major_label_orientation = "vertical"
    p.xgrid.grid_line_color = None
    p.y_range.start = 0

    return p


In [13]:
def make_all_plots(dataframe):
    benchmarks = list(dataframe['host'].unique())
    platforms = list(dataframe['uuid'].unique())
    features = list(dataframe.columns.values)
    features.remove('host')
    features.remove('uuid')
    dataframes = []
    plts = []

    for s in platforms:
    #for l in benchmarks:
        df = pd.DataFrame()
        df = dataframe[dataframe['uuid'].astype('str') == s]
        if len(df) != 0:
            nunique = df.nunique()
            cols_to_update = nunique[nunique == 1].index
            for col in cols_to_update:
                if col != 'host' and col != 'uuid': # need change this to check whether feature is str type
                    random_vector = 0.000000001 * np.random.randint(low = 1, high = 3, size = len(df[col]))
                    df[col] += random_vector
            df = df.drop(['host', 'uuid'], axis=1)
            dataframes.append(df)  
            
    plts.append(interactive_correlation_heatmap(dataframes, benchmarks, features))
    plts.append(interactive_scatterplot(dataframes, benchmarks, features))
    plts.append(interactive_histgram(dataframes, benchmarks, features))
    plts.append(interactive_boxplot(dataframes, benchmarks, features, log_transformation = True))        
    
    y = dataframe['host']
    dataframe = dataframe.drop(['host', 'uuid'], axis=1)
    uncorr_df = removeCorrelatedFeatures(dataframe)
    selectedFeatures = []
    
    fig1 = selectFeaturesChart(uncorr_df, y, selectedFeatures, "mutual_info_classif")
    fig2 = selectFeaturesChart(uncorr_df, y, selectedFeatures, "chi2")
    fig3 = selectFeaturesChart(uncorr_df, y, selectedFeatures, "f_classif")
    
    pre = PreText(text = """Final Selected Features:""",width=1000, height=10)
    p1 = Paragraph(text = str(np.unique(selectedFeatures)), width=1000, height=30)
    p2 = Paragraph(text = f"In total {len(selectedFeatures)} features being selected", width=1000, height=60)
    div = Div(text = f"""
    The best {len(selectedFeatures)} features being selected after filter out high correlation features, low variance features, and combination of information metrics from <i>mutual_info_classif</i>, <i>chi2</i>, <i>f_classif</i>.
    <ul>
    <li><i>mutual_info_classif</i> is mutual information estimation for a discrete target variable.</li>
    <li><i>chi2</i> compute chi-squared stats between each non-negative feature and class.</li>
    <li><i>f_classif</i> compute the ANOVA F-value for the provided sample.</li>
    </ul>
    Refer the <a href="https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection">Sklearn API</a> for more information""", width=1000, height=80)
    
    plts.append(TabPanel(child = column(children = [row(fig1, fig2, fig3), pre, p1, p2, div]), title="Feature Selection"))
    
    show(Tabs(tabs=plts))
    output_file("EDA.html")

In [14]:
#df = load_excel_to_dataframe()
df = load_excel_to_dataframe(pwd + inputDir + filename)
df = drop_string_datetime_na(df)
make_all_plots(df)

C:\Users\42times28\AppData\Local\Temp\ipykernel_12316\3322588224.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[dataframe.T[dataframe.dtypes==np.int64].index] = dataframe[dataframe.T[dataframe.dtypes==np.int64].index].astype(float)
C:\Users\42times28\AppData\Local\Temp\ipykernel_12316\1982214454.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] += random_vector
C:\Users\42times28\AppData\Local\Temp\ipykernel_12316\1982214454.py:20: SettingWithCopyWarning: 
A value is trying to be s